In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,47075
2,Huelva,Confirmados PDIA 14 días,667
3,Huelva,Tasa PDIA 14 días,"129,97642106904144"
4,Huelva,Confirmados PDIA 7 días,356
5,Huelva,Tasa PDIA 7 dias,"69,37272248962333"
6,Huelva,Total Confirmados,47279
7,Huelva,Curados,45640
8,Huelva,Fallecidos,423


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  47075.0


/tmp/ipykernel_87564/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13014.0


/tmp/ipykernel_87564/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_87564/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_87564/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_87564/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 972 personas en los últimos 7 días 

Un positivo PCR cada 547 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,47075.0,356.0,667.0,513170.0,69.372722,129.976421,132.0
Huelva-Costa,27833.0,218.0,398.0,289548.0,75.289762,137.455620,72.0
Huelva (capital),13014.0,148.0,263.0,143837.0,102.894248,182.845860,49.0
Condado-Campiña,14669.0,123.0,233.0,156231.0,78.729574,149.138135,49.0
Moguer,2087.0,27.0,45.0,21867.0,123.473728,205.789546,12.0
Sierra de Huelva-Andévalo Central,4144.0,14.0,33.0,67391.0,20.774287,48.967963,10.0
Cartaya,2221.0,34.0,67.0,20083.0,169.297416,333.615496,10.0
Palma del Condado (La),1466.0,17.0,27.0,10801.0,157.392834,249.976854,8.0
Palos de la Frontera,1150.0,28.0,58.0,11742.0,238.460228,493.953330,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),10.0,4.0,5.0,243.0,1646.090535,2057.613169,3.0
Palos de la Frontera,1150.0,28.0,58.0,11742.0,238.460228,493.953330,6.0
Cartaya,2221.0,34.0,67.0,20083.0,169.297416,333.615496,10.0
Paterna del Campo,222.0,7.0,10.0,3457.0,202.487706,289.268152,4.0
San Bartolomé de la Torre,398.0,4.0,10.0,3761.0,106.354693,265.886732,1.0
Palma del Condado (La),1466.0,17.0,27.0,10801.0,157.392834,249.976854,8.0
Moguer,2087.0,27.0,45.0,21867.0,123.473728,205.789546,12.0
Alosno,240.0,6.0,8.0,3933.0,152.555301,203.407068,4.0
Bollullos Par del Condado,1339.0,12.0,28.0,14387.0,83.408633,194.620143,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lepe,3057.0,1.0,7.0,27880.0,3.586801,25.107604,0.0,0.142857
Isla Cristina,3194.0,5.0,19.0,21393.0,23.372131,88.814098,0.0,0.263158
San Bartolomé de la Torre,398.0,4.0,10.0,3761.0,106.354693,265.886732,1.0,0.400000
Sierra de Huelva-Andévalo Central,4144.0,14.0,33.0,67391.0,20.774287,48.967963,10.0,0.424242
Bollullos Par del Condado,1339.0,12.0,28.0,14387.0,83.408633,194.620143,5.0,0.428571
Almonte,2365.0,10.0,22.0,24507.0,40.804668,89.770270,3.0,0.454545
Palos de la Frontera,1150.0,28.0,58.0,11742.0,238.460228,493.953330,6.0,0.482759
Gibraleón,967.0,8.0,16.0,12737.0,62.809139,125.618277,5.0,0.500000
Encinasola,164.0,1.0,2.0,1310.0,76.335878,152.671756,0.0,0.500000
